# 서울시 공원 데이터

In [5]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 서울시 주요 공원현황.csv to 서울시 주요 공원현황 (1).csv


In [29]:
import folium
import numpy as np
import matplotlib as plt

In [12]:
import pandas as pd
df = pd.read_csv(filename, encoding='euc-kr')
df.head()

,연번,공원명,공원개요,면적,개원일,주요시설,주요식물,안내도,오시는길,이용시참고사항,이미지,지역,공원주소,관리부서,전화번호,X좌표(GRS80TM),Y좌표(GRS80TM),X좌표(WGS84),Y좌표(WGS84),바로가기
0,1,남산도시자연공원,남산공원은 도심에 위치하여 서울시민에게 맑은 공기를 제공하는 자연휴식처이며 산책 꽃...,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡,1968.9.10,기반시설 : 광장 45950㎡ 도로 108530㎡ 산책로 6.7㎞ (북측:3.7㎞/...,소나무 단풍 아카시아 상수리나무 등 191종 2881870주,http://parks.seoul.go.kr/upload/seditorMulti/2...,명동역(남산케이블카 와룡묘 서울애니메이션센터 방면) 도보 명동역 3번출구 → 퍼시픽...,남산공원 차량통행 금지안내 2005년 5월 1일부터 남산공원에 일반승용차 택시 통행...,http://parks.seoul.go.kr/file/info/view.do?fId...,중구,서울특별시 중구 삼일대로 231(예장동),서울특별시중부푸른도시사업소,02-3783-5900,198364.107,450395.554,126.990377,37.550140,http://parks.seoul.go.kr/template/sub/namsan.do
1,2,길동생태공원,길동생태공원은 생물의 서식처를 제공하고 종다양성을 증진시키며 자연생태계의 생물들을 ...,80683㎡,1999.5.20,탐방객안내소 야외전시대 관찰대 목재데크 조류관찰대 길동생태문화센터,소나무 보리수 등 64종 31800주 산국 부들 등 138종 192800본,http://parks.seoul.go.kr/template/common/img/p...,NaN,매주 월요일은 휴관입니다 사전예약 후 입장하실 수 있습니다. ▶ 인터넷 예약 바로가...,http://parks.seoul.go.kr/file/info/view.do?fId...,강동구,서울특별시 강동구 천호대로 1291(길동생태공원),동부공원녹지사업소 길동생태공원,02-489-2770,213554.120,448852.675,127.154779,37.540394,http://parks.seoul.go.kr/template/sub/gildong.do
2,3,서울대공원,서울대공원은 세계 각국의 야생동물들이 살아 숨 쉬는 서울동물원과 다양한 재미와 즐거...,9132690m²,1984.5.1,동물원 식물원 테마가든(장미원 어린이동물원 피크닉장) 치유숲 산림욕장 캠핑장 국립현...,NaN,NaN,지하철 4호선 대공원역 하차(2번 출구) 도보 15분 정도 문의처 02)500-73...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,과천시,경기도 과천시 대공원광장로 102,전략기획실,02-500-7032,200994.267,437163.981,127.019847,37.426449,http://grandpark.seoul.go.kr/
3,4,서울숲,당초 골프장 승마장 등이 있던 뚝섬일대를 주거업무 지역으로 개발할 경우 약 4조원에...,480994㎡,2005.6.18,4개 테마공원: 문화예술공원 자연생태숲 자연체험학습원 습지생태원 주요시설: 가족마당...,수 목 : 소나무 섬잣나무 계수나무 외 95종 415795주 식물원 : 선인장 등 ...,http://parks.seoul.go.kr/template/common/img/p...,NaN,지하철 분당선 서울숲역 3번출구 도보로 약5분 2호선 뚝섬역 8번출구 도보로 약 1...,http://parks.seoul.go.kr/file/info/view.do?fId...,성동구,서울특별시 성동구 뚝섬로 273 (성수동1가),서울숲 관리사무소,02-460-2905,203695.432,449290.726,127.041798,37.543072,http://parks.seoul.go.kr/template/sub/seoulfor...
4,5,월드컵공원,월드컵공원은 서울의 서쪽에 위치하여 1978년부터 1993년까지 15년간 서울시민이...,2284085㎡,2002.5.1,평화의 공원 월드컵공원전시관(879㎡) 유니세프광장(2400㎡) 평화광장(5217㎡...,NaN,http://parks.seoul.go.kr/template/common/img/p...,지하철 6호선 월드컵경기장역 하차 → 1번 출구로 나온 후 직진 → 큰길(도로)이 ...,일반사항 공원을 깨끗하게 이용합니다. 대중교통을 이용해 주세요. 기념물 시설물 풀과...,http://parks.seoul.go.kr/file/info/view.do?fId...,마포구,서울특별시 마포구 하늘공원로 84(월드컵공원),월드컵공원,02-300-5500,190658.070,451598.831,126.878907,37.571805,http://parks.seoul.go.kr/template/sub/worldcup...


In [22]:
df.columns

Index(['연번', '공원명', '공원개요', '면적', '개원일', '주요시설', '주요식물', '안내도', '오시는길',
       '이용시참고사항', '이미지', '지역', '공원주소', '관리부서', '전화번호', 'X좌표(GRS80TM)',
       'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '바로가기'],
      dtype='object')

In [27]:
area_df[['공원명', '면적']]

,공원명,면적
0,남산도시자연공원,2896887㎡ 임 야 : 2454140㎡ 녹지대 및 기타시설 : 442747㎡
1,길동생태공원,80683㎡
2,서울대공원,9132690m²
3,서울숲,480994㎡
4,월드컵공원,2284085㎡
...,...,...
127,용마도시자연공원(사가정공원),109635㎡
128,문화비축기지,140022㎡
129,경춘선숲길,211392㎡
130,율현공원,157535㎡


In [32]:
area = df.면적.astype(str).apply(lambda x: x.strip('총 ').split()[0])
area[:-5]

0       2896887㎡
1         80683㎡
2      9132690m²
3        480994㎡
4       2284085㎡
         ...    
122       25500㎡
123        2560㎡
124       32000㎡
125       45615㎡
126      631021㎡
Name: 면적, Length: 127, dtype: object

In [35]:
area[(area == 'nan') | (area == '휴양')]

6       휴양
109    nan
119    nan
131    nan
Name: 면적, dtype: object

In [36]:
area[(area == 'nan') | (area == '휴양')] = '0㎡'
area[-5:]

127    109635㎡
128    140022㎡
129    211392㎡
130    157535㎡
131         0㎡
Name: 면적, dtype: object

In [37]:
import re
area = area.apply(lambda x: float(re.sub('[㎡m²㎥]','',x)))

In [38]:
area = area.apply(lambda x: int(np.round(x)))

## 2. 공원의 면적에 따라 분류, 크기

In [39]:
area_criteria = [-1, 100000, 1000000, 12000000]
labels = ['소형', '중형', '대형']
size_info = [3, 7, 15]
scale = pd.cut(area, area_criteria, labels=labels)
size = pd.cut(area, area_criteria, labels=size_info)
scale[:5]

0    대형
1    소형
2    대형
3    중형
4    대형
Name: 면적, dtype: category
Categories (3, object): ['소형' < '중형' < '대형']

In [40]:

size[:5]

0    15
1     3
2    15
3     7
4    15
Name: 면적, dtype: category
Categories (3, int64): [3 < 7 < 15]

## 3. 새로운 데이터 프레임 만들고 저장

In [42]:
df = df[['공원명','지역','X좌표(WGS84)','Y좌표(WGS84)']]
df.columns = ['공원명','지역','경도','위도']
df['면적'] = area
df['분류'] = scale
df['크기'] = size
df.head()

,공원명,지역,경도,위도,면적,분류,크기
0,남산도시자연공원,중구,126.990377,37.550140,2896887,대형,15
1,길동생태공원,강동구,127.154779,37.540394,80683,소형,3
2,서울대공원,과천시,127.019847,37.426449,9132690,대형,15
3,서울숲,성동구,127.041798,37.543072,480994,중형,7
4,월드컵공원,마포구,126.878907,37.571805,2284085,대형,15


In [43]:

df.isnull().sum()

공원명    0
지역     1
경도     1
위도     1
면적     0
분류     0
크기     0
dtype: int64

In [44]:
df[df.경도.isna()]

,공원명,지역,경도,위도,면적,분류,크기
131,서울로7017,중구,NaN,NaN,0,소형,3


In [45]:
df = df.dropna()

In [46]:
df.to_csv('서울공원요약.csv', index=False)

## 4. 공원 시각화

In [47]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11)
for i in df.index:
    folium.CircleMarker([df.위도[i], df.경도[i]], 
                        radius=int(df.크기[i]),
                        tooltip=f"{df.공원명[i]}({df.면적[i]:,d}㎡)",
                        color='crimson', fill_color='crimson').add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>서울시내 공원 위치와 크기</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map

In [48]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=11, tiles='Stamen Toner')
for i in df.index:
    folium.CircleMarker([df.위도[i], df.경도[i]], 
                        radius=int(df.크기[i]),
                        tooltip=f"{df.공원명[i]}({df.면적[i]:,d}㎡)",
                        color='green', fill_color='green').add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>서울시내 공원 위치와 크기</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map